# Tối ưu hóa Chiến thuật Bóng đá: Tiếp cận bằng Random Forest

## 1. Giới thiệu & Lý do chọn Mô hình
Trong notebook này, chúng ta sẽ nâng cấp từ mô hình cơ bản (Naive Bayes) lên một mô hình mạnh mẽ hơn là **Random Forest (Rừng ngẫu nhiên)**.

### Tại sao Random Forest mạnh hơn?
1.  **Xử lý Quan hệ Phi tuyến tính:** Bóng đá rất phức tạp. Kiểm soát bóng nhiều chưa chắc đã thắng nếu không sút. Random Forest hiểu được các mối quan hệ "IF-THEN" phức tạp này (Interaction effects), điều mà Naive Bayes (vốn giả định các biến độc lập) không làm được.
2.  **Chống Overfitting:** Bằng cách kết hợp kết quả của hàng trăm cây quyết định (Decision Trees), nó giảm thiểu sai số phương sai.
3.  **Feature Importance:** Nó cho biết chính xác yếu tố nào (xG, Pressing, hay Passing?) quan trọng nhất để giành chiến thắng.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

## 2. Chuẩn bị Dữ liệu

In [ ]:
df = pd.read_csv('tactical_dataset_with_results.csv')

features = ['possession_ratio', 'num_passes', 'avg_pass_length', 'shots', 'xg', 
            'pressures', 'tackles', 'interceptions', 'avg_x_position', 
            'team_width', 'final_third_actions', 'ppda']
target = 'result'

# Chia tập dữ liệu (Stratified Split)
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## 3. Huấn luyện Random Forest
Chúng ta sử dụng `class_weight='balanced'` để hỗ trợ mô hình nhận diện tốt hơn các lớp ít mẫu (như Draw).

In [ ]:
# Khởi tạo mô hình
rf_model = RandomForestClassifier(
    n_estimators=100,      # Số lượng cây: 100
    max_depth=10,          # Độ sâu tối đa để tránh học vẹt (overfitting)
    class_weight='balanced', # Tự động cân bằng trọng số cho class Draw
    random_state=42,
    n_jobs=-1              # Sử dụng tất cả CPU core
)

# Train
rf_model.fit(X_train, y_train)

## 4. Đánh giá Hiệu năng

In [ ]:
y_pred = rf_model.predict(X_test)

print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
plt.figure(figsize=(8,6))
cm = confusion_matrix(y_test, y_pred, labels=rf_model.classes_)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', 
            xticklabels=rf_model.classes_, yticklabels=rf_model.classes_)
plt.title('Confusion Matrix - Random Forest')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

## 5. Phân tích Tầm quan trọng của Đặc trưng (Feature Importance)
Đây là bước quan trọng nhất để "Đề xuất chiến thuật". Chúng ta cần biết yếu tố nào quyết định trận đấu.

In [ ]:
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, 6))
plt.title("Feature Importances (Mức độ ảnh hưởng đến kết quả)")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), [features[i] for i in indices], rotation=45)
plt.tight_layout()
plt.show()

# In top 5 quan trọng nhất
print("Top 5 Key Factors:")
for i in range(5):
    print(f"{i+1}. {features[indices[i]]} ({importances[indices[i]]:.4f})")

## 6. Phân tích DNA của đội chiến thắng
Thay vì dùng giá trị trung bình (dễ bị nhiễu), ta dùng **Boxplot** để xem phân phối của các chỉ số quan trọng trong các trận Thắng.

In [ ]:
# Lấy top 3 features quan trọng nhất
top_3_features = [features[i] for i in indices[:3]]

plt.figure(figsize=(15, 5))
for i, col in enumerate(top_3_features):
    plt.subplot(1, 3, i+1)
    sns.boxplot(x='result', y=col, data=df, order=['Win', 'Draw', 'Loss'], palette="Set2")
    plt.title(f'Phân phối {col} theo Kết quả')
plt.tight_layout()
plt.show()

## 7. Hệ thống Đề xuất Chiến thuật Thông minh
Hệ thống so sánh chỉ số của đội bóng với mức Median (Trung vị) của các đội Chiến thắng.

In [ ]:
def smart_tactical_advice(current_stats_dict, df_data, top_features):
    # Lấy dữ liệu các trận thắng
    win_df = df_data[df_data['result'] == 'Win']
    
    print("--- PHÂN TÍCH & KHUYẾN NGHỊ ---")
    
    for feature in top_features:
        current_val = current_stats_dict.get(feature, 0)
        # Mục tiêu là đạt mức 50% (median) của đội thắng
        target_val = win_df[feature].median()
        
        diff = current_val - target_val
        
        status = "OK"
        advice = ""
        
        if feature == 'ppda': # PPDA càng thấp càng tốt (pressing mạnh)
            if current_val > target_val * 1.2:
                advice = f"Cần PRESSING mạnh hơn. Giảm PPDA xuống {target_val:.1f}"
            else:
                advice = "Cường độ Pressing tốt."
        else: # Các chỉ số tấn công (xG, Shots, Possession...) càng cao càng tốt
            if current_val < target_val * 0.9:
                advice = f"Cần TĂNG cường độ. Mục tiêu: {target_val:.2f}"
            else:
                advice = "Đạt yêu cầu."
                
        print(f"> {feature.upper()}: Hiện tại={current_val:.2f} | Mục tiêu={target_val:.2f} -> {advice}")

# Demo với dữ liệu mẫu (giả sử một trận thua)
sample_loss = df[df['result'] == 'Loss'].iloc[0]
current_stats = sample_loss[features].to_dict()

smart_tactical_advice(current_stats, df, [features[i] for i in indices[:5]])